# NVT模拟退火
先模拟退火，再成品模拟

In [ ]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as mda
import nglview as nv
import numpy as np

In [ ]:
# 与处理蛋白
fixer=PDBFixer(pdbid='5xg0')
fixer.removeChains(chainIds=["B","C"])
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(keepWater=False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*angstroms,positiveIon='Na+',negativeIon='Cl-')
PDBFile.writeFile(fixer.topology,fixer.positions,open('5xg0_process.pdb','w'))

In [ ]:
# 查看生成的pdb文件
view=nv.show_mdanalysis(mda.Universe('5xg0_process.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

In [ ]:
# 模拟设置 
pdbfile='5xg0_process.pdb' # 处理好的pdb文件

nonbondedmethod=PME   # 处理非相互作用力 NoCutoff | CutoffNonPeriodic | CutoffPeriodic | Ewald | PME | LJPME 
nonbondedcutoff=1*nanometer  # 非键相互作用的阶段距离
constraints= HBonds   # 施加约束 None, HBonds, AllBonds, or HAngles
removecmmotion=True    # 是否移除质心运动
rigidwater = True      # 是否使用刚性水
ewaldErrorTolerance = 0.0005  
constraintTolerance = 0.000001     
hydrogenMass = 1.5*amu   # 氢质量

dt = 0.002*picoseconds   # 步长
temperature = 300*kelvin   # 模拟温度
friction = 1.0/picosecond  # 温度耦合时间
pressure = 1.0*atmospheres  # 压力
barostatInterval = 25       # 压力耦合步数

annealtemp=np.array([300,280,300,320,340,360,340,320,300])   # 模拟退火温度设置
annealstep=10000     # 每个温度步数

equilibrationSteps=10000    # 平衡步长
steps=20000                # 模拟步长
 
dcd=True                    # 是否保存dcd文件
dcdstep=1000                # 多少步保存一次轨迹
chk=True                    # 是否保存检查点文件
chkstep=1000                # 多少步保存一次检查点文件
data=stdout               # 以文本格式还是直接输出文件 stdout | 'log.txt
datastep=100               # 多少步输出一次运行信息

dcdReporter = DCDReporter('trajectory.dcd', dcdstep)
dataReporter = StateDataReporter(data,datastep, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True,density=True,volume=True,separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', chkstep)

platform=Platform.getPlatformByName("CUDA")  # 模拟平台 CUDA | CPU |OPENGL
properties={"DeviceIndex":"0","Precision":"single"}  # 选择cuda时所使用的显卡，以及需要的精度类型 single double

In [ ]:
pdb=PDBFile(pdbfile)
forcefield=ForceField('amber14-all.xml','amber14/tip3p.xml')
system=forcefield.createSystem(pdb.topology,nonbondedMethod=nonbondedmethod,
                                nonbondedCutoff=nonbondedcutoff,
                                constraints=constraints,
                                removeCMMotion=removecmmotion,
                                rigidWater=rigidwater,
                                ewaldErrorTolerance=ewaldErrorTolerance, 
                                hydrogenMass=hydrogenMass
                                )


In [ ]:
integrator=LangevinIntegrator(temperature,friction,dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation=Simulation(pdb.topology,system,integrator,platform)
simulation.context.setPositions(pdb.positions)

In [ ]:
# 最小化
simulation.minimizeEnergy()

In [ ]:
# 平衡
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

In [ ]:
simulation.currentStep = 0

for i in annealtemp:
    integrator.setTemperature(i)
    simulation.step(annealstep)


In [ ]:
# 模拟
if dcd:
    simulation.reporters.append(dcdReporter)
else:
    pass
simulation.reporters.append(dataReporter)
if chk:
    simulation.reporters.append(checkpointReporter)
else:
    pass
simulation.currentStep = 0
simulation.step(steps)
positions=simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology,positions,open("lastframe.pdb","w"))

In [ ]:
# 查看生成的pdb文件
view=nv.show_mdanalysis(mda.Universe('lastframe.pdb'))
view.add_licorice('water',opacity=0.35)
view.add_spacefill('ion')
view.center('all')
view

In [ ]:
traj=nv.show_mdanalysis(mda.Universe('lastframe.pdb','trajectory.dcd'))
# traj.add_licorice('water',opacity=0.35)
# traj.add_spacefill('ion')
traj